In [1]:
# importing required modules

# importing xlrd module this module helps us in reading the excel files
import xlrd

# importing xlwt module this module helps us to write in excel files
import xlwt

# from xutils.copy importing copy this helps us to copy the existing excel file and overwrite this file
from xlutils.copy import copy

# importing getpass module this module helps us to take password in hidden form
import getpass

# from time importing sleep module this stops the program for some seconds
from time import sleep

# importing sys module it helps us to exit from program
import sys


# these are global variable these variables used in mostly every function

# opening excel workbook
bank = xlrd.open_workbook("bank.xls")

# opening excel sheet 
bank_sheet = bank.sheet_by_index(0)

# creating user_details variable as a dictionary it stored username as a key and all rest details in a form of list
# as a value it manages the details of the user
user_details = {}

# creating admin details dictionary
admin_details = {}

# getting another sheet where admin details are present
bank_sheet_2 = bank.sheet_by_index(1)

for i in range(bank_sheet_2.nrows):
    admin_details[bank_sheet_2.cell_value(i,0)] = bank_sheet_2.cell_value(i,1)

# storing user details in a dictionary using for loops
for i in range(1,bank_sheet.nrows):
    k = ""
    l = []
    for j in range(bank_sheet.ncols):
        if j == 0:
            k = bank_sheet.cell_value(i,j)
            continue
        if j == 2:
            l.append(int(bank_sheet.cell_value(i,j)))
            continue
        if j == 3:
            l.append(float(bank_sheet.cell_value(i,j)))
            continue
        l.append(bank_sheet.cell_value(i,j))
    user_details[k] = l


# creating class ATM where ATM functions is defined
class ATM():
    
#     this functon update the excel sheet
    def update(self):
        
        '''this function updates the excel sheet according to the user_details'''
        
#         initializing required global variables
        global bank , bank_sheet , user_details , admin_details
        
#         creating workbook
        new_bank = xlwt.Workbook()
    
#         creating sheet inside workbook
        new_bank_sheet = new_bank.add_sheet("Sheet 1")
    
#     writing data
        new_bank_sheet.write(0,0,"Username")
        new_bank_sheet.write(0,1,"Name")
        new_bank_sheet.write(0,2,"Pin")
        new_bank_sheet.write(0,3,"Amount")
        
#         initializing index variable it helps loop to increment rows
        index = 1
        
#         running for loop and writing data in excel sheet
        for i in user_details.keys():
            
            new_bank_sheet.write(index,0,i)
            new_bank_sheet.write(index,1,user_details[i][0])
            new_bank_sheet.write(index,2,user_details[i][1])
            new_bank_sheet.write(index,3,user_details[i][2])
            index+=1
            
#         creating sheet 2 and adding admin username and password
        new_bank_sheet_2 = new_bank.add_sheet("Sheet 2")
        index2 = 0
        for i,j in admin_details.items():
            new_bank_sheet_2.write(index2,0,i)
            new_bank_sheet_2.write(index2,1,j)
            index2+=1
        
#         saving created excel workbook
        new_bank.save("bank.xls")
        
    
#     this function update data of user_details
    def update_data(self):
        
        '''this function reading data from excel sheet and update the user details'''
        
#         calling required global variables
        global bank , bank_sheet , user_details , admin_details
        
#         opening and reading workbook
        bank = xlrd.open_workbook("bank.xls")
        bank_sheet = bank.sheet_by_index(0)
        user_details = {}
        bank_sheet_2 = bank.sheet_by_index(1)
        admin_details = {}
        for i in range(bank_sheet_2.nrows):
            admin_details[bank_sheet_2.cell_value(i,0)] = bank_sheet_2.cell_value(i,1)

#         adding read data in user_details
        for i in range(1,bank_sheet.nrows):
            k = ""
            l = []
            for j in range(bank_sheet.ncols):
                if j == 0:
                    k = bank_sheet.cell_value(i,j)
                    continue
                if j == 2:
                    l.append(int(bank_sheet.cell_value(i,j)))
                    continue
                if j == 3:
                    l.append(float(bank_sheet.cell_value(i,j)))
                    continue
                l.append(bank_sheet.cell_value(i,j))
            user_details[k] = l
        
        
      
    
#     this function checking pin
    def pin(self , username , tries):
        
        '''this function matches pin this function contains two parameter 1st parameter is username it helps function
        to match pin using username and 2nd parameter in number of tries initially tries is equals to 1 this function
        also handles various exceptions'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             if tries greater than 3 this means user exceed maximum 3 tries and it returns to the main function
            if tries > 3:
                
                print("You Exceed maximum tries")
                return main()
            
#             getting pin in hidden form
            pin = int(getpass.getpass(prompt="Enter Pin : "))
            
#             if pin matches then it returns to the user function for further options
            if pin == user_details[username][1]:
                
                return self.user(username)
            
#             else if pin not matches this means that pin is incorrect and it return to the same function 
#             with argument username and tries by increment 1
            else:
                
                print("Invalid Pin")
                return self.pin(username ,tries+1)
         
        
#         handling exceptions
        except KeyboardInterrupt:
            print("Program Stopped by backend")
            
        except SystemExit:
            print("Exitted from system")
          
        
#         if any other type of error arise this means that pin is not in correct format then it returns to the same function
#         with argument username and tries by increment 1
        except:
            print("Pin must be an integer")
            return self.pin(username , tries+1)
        
            
            
#     this function maintaing user part
    def user(self , username):
        
        '''this function mainting user part and printing user service options and calling various required functions
        this function contains 1 parameter username it helps to get user details this function handling various exception'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             printing user name and user account balance
            print("Welcome "+user_details[username][0])
            print("Account Balance : "+str("Rs. {:.2f}".format(user_details[username][2])))
            
#             creating dictionary of various service functions
            options = {1:self.Cash_deposit 
                       , 2:self.Cash_withdrawl , 
                       3:self.New_pin_generate , 
                       4:self.Sign_out}

#             printing various services
            for i,j in options.items():
                print(str(i)+". "+" ".join(j.__name__.split("_")))

#             taking input from user to choose the service
            opt = int(input("Choose service from above options : "))

#             checking if choosen option present in options or not
            if opt in options:
                
#                 if choosen option is equals to 2 or 3 this means that user calling cash withdrwal or new pin generate
#                 with arguments username and number of tries
                if opt == 2 or opt == 3:
                    
                    options[opt](username , 1)
                    
#                 else if choosen option equals to 4 this means than user wants to sign out 
                elif opt == 4:
                    
                    options[opt]()
                 
#                 else it call functions accordingly with argument username
                else:
                    options[opt](username)
              
                
#             else user not choosen correct service and it returns to same function
            else:
                
                print("Choose from above services")
                return self.user(username)
            
            
#         handling different exceptions
        except KeyboardInterrupt:
            print("System Stopped by backend")
            
        except SystemExit:
            print("Exitted from system")
           
        
#         if any other exception occurs this means that user not entered correct option then it returns to same function
#         after giving message
        except:
            print("Input must be an integer")
            return self.user(username)
        
        
         
            
#     this function deposits the cash
    def Cash_deposit(self , username):
        
        '''this function deposits the cash to the user account this function contains 1 parameter which is 
        username this helps function to get user account balance this function also handling various 
        exceptions'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             asking user to enter amount which user wants to deposit
            cash = float(input("Enter amount : "))
            
#             if amount less than or equal to zero then it returns to the same function because balance can not be in negative
#             or zero
            if cash <= 0:
                print("Amount not be negative")
                return self.Cash_deposit(username)
            
#             updating user balance in excel sheet and user details
            user_details[username][2]+=cash
            self.update()
            self.update_data()
            
#             printing confirmation message
            print("Cash deposited successfully")
            sleep(2)
            
#             returning to the user function
            return self.user(username)
        
#         handling various exceptions
        except KeyboardInterrupt:
            print("System Stopped by backend")
            
        except SystemExit:
            print("Exitted from system")
            
            
#         if any other error arise this means that user not entered correct amount then it returns to the same function
#         after giving warning message
        except:
            print("Input must be float or integer")
            return self.Cash_deposit(username)
        
    
#     this function withdraw cash
    def Cash_withdrawl(self , username , tries):
        
        '''this function withdraw cash from user account this function contains 2 parameter 1st parameter is
        username which helps function to fetch user balance and update it and 2nd parameter is number of tries
        which is initially 1 this function also handling various exceptions'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             if tries greater than 3 this means that user exceed maximum tries and it returns to the main function
            if tries > 3:
                print("You exceed maximum tries login again")
                return main()
            
#             taking old pin from the user to withdraw amount in hidden format
            access_pin = int(getpass.getpass(prompt="Enter pin to withdraw amount : "))
            
#             if old pin not matches then it returns to same function with argument username and tries increment by 1
            if access_pin != user_details[username][1]:
                print("Incorrect pin")
                return self.Cash_withdrawl(username , tries+1)
            
#             asking amount to withdraw from the user
            cash = float(input("Enter amount : "))
            
#             if amount less than or equal to zero then it returns to the same function with username and tries
#             because amount can not be in negative
            if cash < 0:
                print("Amount not be negative")
                return self.Cash_withdrawl(username ,tries)
            
#             else if after withdraw amount it is in negative this means insufficient balance then it returns to the same
#             function with argument username and tries
            elif user_details[username][2]-cash < 0:
                print("Insufficient balance")
                return self.Cash_withdrawl(username , tries)
            
#             this line execute this means sufficient balance and update balance in excel sheet and user details
            user_details[username][2]-=cash
            
            self.update()
            self.update_data()
            
#             printing confirmation message
            print("Cash withdrawl successfully")
            sleep(2)
        
#             returning to user 
            return self.user(username)
        
#         handling different exceptions
        except KeyboardInterrupt:
            print("System Stopped by backend")
            
        except SystemExit:
            print("Exitted from system")
          
        
#         if any other exception aries this means that user not entered pin or cash in correct format and it returns to the
#         same function with argument username and tries
        except:
            print("Enter correct input")
            return self.Cash_withdrawl(username , tries)
        
    
    
#     this function generate new pin
    def New_pin_generate(self , username , tries):
        
        '''this function generate new pin of user this function contains 2 parameter username and tries
        username helps function to fetch user details and 2nd parameter contains number of maximum tries
        initially tries equals to 1'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             if tries greater than 3 this means that user exceed maximum tries and returns to the main function
            if tries > 3:
                print("You excced maximum tries login again")
                return main()
            
#             getting old pin as input from user in hidden form
            access_pin = int(getpass.getpass(prompt="Enter old pin : "))
            
#             if old pin not matches then it returns to the same function with arguments username and tries by increment 1
            if access_pin != user_details[username][1]:
                print("incorrect pin")
                return self.New_pin_generate(username , tries+1)
            
#             asking user to enter new pin in hidden format
            new_pin = int(getpass.getpass(prompt="Enter new pin : "))
            
#             if new pin / 1000 less than 1 this means that pin not of 4 digits it returns to the same function with
#             argument username and tries
            if new_pin/1000 < 1:
                print("Pin must be of 4 digits")
                return self.New_pin_generate(username , tries)
            
#             updating user pin in excel sheet and user details
            user_details[username][1] = new_pin
            self.update()
            self.update_data()
            
#             printing confirmation message
            print("Pin changed successfully")
            print("Login again")
            sleep(2)
            
#             returning to the main function
            return main()
        
    
#         handling various functions
        except KeyboardInterrupt:
            print("System Stopped by backend")
        
        except SystemExit:
            print("Exitted from system")
         
        
#         if any other error arise this means that user not entered pin in a correct format and it returns to the same function 
#         with arguments username and tries by increament 1
        except:
            print("Pin must be an integer")
            return self.New_pin_generate(username , tries+1)
        
    
    
#     this function checks admin pin
    def admin_pin(self , username , original_pin , tries):
        
        '''this function matches the admin pin with the correct pin this function contains 2 parameter
        1st parameter is original pin and 2nd parameter is number of tries that user have initially 
        tries equals to 1 this function also handles different types of exception'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             if tries greater than 3 this means that maximum tries exceed and it returns to the main function
            if tries > 3:
                print("You Exceed maximum tries")
                return main()
            
#             getting pin in hidden format
            pin = int(getpass.getpass(prompt="Enter Pin : "))
            
#             if pin matches then it returns to the admin options
            if pin == original_pin:
                return self.admin_options(username)
            
#             elsse it returns to the same function with arguments username and tries by increament 1
            else:
                print("Invalid Pin")
                return self.admin_pin(username , original_pin , tries+1)
          
        
#         handling various exceptions
        except KeyboardInterrupt:
            print("System Stopped by backend")
            
        except SystemExit:
            print("Exitted from system")
         
        
#         if any other exception arise this means that user not entered pin in correct format then it return to 
#         the same function with argument pin and tries increment by 1
        except:
            print("Pin must be an integer")
            return self.admin_pin(username , original_pin , tries+1)
            
      
    
#     this is admin function
    def admin_options(self , username):
        
        '''this is admin function which display all the services and call the functions according to the
        user input this function contains exception handling'''
        
        try:
            
            
#             calling required global variables
            global bank , bank_sheet , user_details , admin_details
            
#         storing functions in a dictionary
            options = {1:self.See_user_details ,
                       2:self.Add_user ,
                       3:self.Remove_user ,
                       4:self.Admin_new_pin_generate,
                       5:self.Sign_out ,
                       6:self.Exit_from_system}

#             printing services
            for i,j in options.items():
                print(str(i)+". "+" ".join(j.__name__.split("_")))


#             asking user to choose the service
            choose = int(input("Enter from above options : "))
            
        
#             if input in optins and user choose equals to 2 this means that admin wants to add new user
#             so it call Add_user function with parameter True
            if choose in options and choose == 2:
                options[choose](username , True)
                
#             else if choose in options and choose equals to 4 this means that user wants to generate
#             new pin so it call Admin_new_pin_generate with argument 1 which is tries
            elif choose in options and choose == 4:
                options[choose](username , 1)
                
#             else if choose in options then it call the function
            elif choose in options and (choose == 1) or (choose == 3):
                options[choose](username)
            
            elif choose in options:
                options[choose]()
                
            
#           else user not choose correct option and it returns to the same function
            else:
                print("Choose correct option")
                return self.admin_options(username)
            
            
#         handling different types of exception
        except KeyboardInterrupt:
            print("System Stopped by backend")
            
        except SystemExit:
            print("Exitted from program")
            
#             if any other type of error arise this means that user not entered correct option then
#             it returns to same function after giving warning
        except:
            print("Input must be an integer")
            return self.admin_options(username)
        
    
    
    
#     this function generate new admin pin
    def Admin_new_pin_generate(self , username , tries):
        
        '''this function genearate new admin pin and update it in excel sheet this function contains 1
        parameter tries which maintains the maximum tries initially tries equal to 1 this function
        also handled different types of exception'''
        
        try:
            
#             importing required global variables
            global bank , bank_sheet , user_details , admin_details
            
        
#             if tries greater than 3 this means that user exceed maximum tries and it returns to the main function
            if tries > 3:
                print("You excced maximum tries login again")
                return main()
            
#             asking user to enter old pin
            access_pin = int(getpass.getpass(prompt="Enter old pin : "))
            
        
#             if old pin not macthes with the right pin then it return to the same function with increment by 1
#         in tries argument
            if access_pin != admin_details[username]:
                print("incorrect pin")
                return self.Admin_new_pin_generate(username , tries+1)
            
#             if pin matches then function coming on this line and asks user to enter new pin
            new_pin = int(getpass.getpass(prompt="Enter new pin : "))
            
        
#         if pin/1000 less than 1 this means that pin less than 4 digits
            if new_pin/1000 < 1:
                print("Pin must be of 4 digits")
                return self.Admin_new_pin_generate(username , tries)
            
#             copying old excel sheet and updating it
            admin_details[username] = new_pin
            self.update()
            self.update_data()
            
#             printing confirmation message
            print("Pin changed successfully")
            print("Login again")
            sleep(2)
            
#             returning to the main function after pin change
            return main()
        
    
#     handling different exceptions
        except KeyboardInterrupt:
            print("System Stopped by backend")
        
        except SystemExit:
            print("Exitted from system")
            
#         if any other error arise this means that user entered wrong pin format then it returns to same 
#         function after increment tries by 1
        except:
            print("Pin must be an integer")
            return self.Admin_new_pin_generate(username , tries+1)
        
     
    
    
#     this function takes new user pin
    def new_user_pin(self):
        
        '''this function takes new user pin in a correct format and also handled different types of exception'''
        
        try:
            
#             taking pin as input from user
            p = int(input("Enter pin : "))
            
#             if pin less than 0 this means that user entered incorrect pin then it returns to the same function
#             after giving warning
            if p<0:
                print("Pin must be greater than zero")
                return self.new_user_pin()
            
#             if pin/1000 less than 1 this means that pin less than 4 digits
            elif p/1000 < 1:
                print("Pin must be of 4 digits")
                return self.new_user_pin()
            
#             returning pin
            return p
        
    
#     handling different exceptions
        except KeyboardInterrupt:
            print("Program stopped by backend")
            
        except SystemExit:
            print("Exitted from system")
            
            
#             if any other error arises this means that pin not in a correct format then it again calls same function
#             after giving warning
        except:
            print("Pin must be an integer")
            return self.new_user_pin()
        
     
    
    
    
#     this function take balance in a correct format
    def new_user_balance(self):
        
        '''this takes takes new user account balance as input in a correct format and return the account balance'''
        
        try:
            
#             taking input 
            p = float(input("Enter balance : "))
            
#             if input less than zero then it again calls the same function because account balance cannot be in negative
            if p<0:
                print("Balance must be greater than zero")
                return self.new_user_balance()
            
#             returning account ba;ance
            return p
        
#     handling error
        except KeyboardInterrupt:
            print("Program stopped by backend")
          
        
        except SystemExit:
            print("Exitted from system")
           
        
#         if any other error occurs this means that user not entered balance in a correct format then it
#         returns to the same function after giving warning
        except:
            print("Balance must be an integer or float")
            return self.new_user_balance()
        
    
    
#     this function add user to the excel sheet
    def Add_user(self , username , is_admin):
        
        '''this function add user to excel sheet it takes 1 parameter whether user is admin or not 
        and give options accordingly this function also includes exception handling if user press stop button
        then keyboardinterrupt arise it exit from the program after giving message and if systemexit error
        arise this means that sys.exit() function used it exits from the program and if any other type of error
        come then it returns to the main function'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
            print("Enter details for new user")

#             to add new user user details first it copy old excel sheet
            new_bank = copy(bank)
            new_bank_sheet = new_bank.get_sheet(0)

#             by using for loop it asks user details and adds in the excel sheet
            for j in range(bank_sheet.ncols):

#                 if j equals to 2 this means that program asks to user to enter the pin and to take pin in a 
#                 correct format it calls the function new_user_pin
                if j == 2:
                    p = self.new_user_pin()
                    new_bank_sheet.write(bank_sheet.nrows,j,p)
                    continue

#                     if j equals to 3 this means that program asks to user to enter the acoount balance and to 
#                     take account balance in a correct format it calls the function new_user_balance
                elif j == 3:
                    new_bank_sheet.write(bank_sheet.nrows,j,self.new_user_balance())
                    continue

                new_bank_sheet.write(bank_sheet.nrows,j,input("Enter {} : ".format(bank_sheet.cell_value(0,j))))

#             and at the end it saves the new file and update the data
            new_bank.save("bank.xls")
            self.update_data()
        
#         printing confirmation message
            print("User Added Successfully")
            
        
#             if user is admin then it asks to the admin to go back to the main menu or sign out
            if is_admin:

                opt = int(input("Press 0 to return to the main menu else press any other key to signout : "))

                if opt == 0:
                    return self.admin_options(username)

                else:
                    return main()
              
            
#             else it sign out and return to the main function
            else:
                return main()
            
        except KeyboardInterrupt:
            print("System Stopped by backend")
        
        except SystemExit:
            print("Exitted from system")
            
        except:
            return main()
        
        
        
#     this function remove user form the excel sheet
    def Remove_user(self , username):
        
        '''this function remove user from the excel sheet this function can only accessed by admin because 
        only admin have rights to add or remove user this function uses exception handling if user press
        stop button then keyboardinterrupt error arise and it exit from the program after giving warning
        message to the user if systemexit error arise this means that sys.exit() function used
        this function first prints the user details to the admin and asks from the admin which user
        admin wants to remove is user present in user_details then it detele the user and update the excel sheet'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#         printing user details to the admin
            for i in range(bank_sheet.nrows):

                for j in range(bank_sheet.ncols):

                    if i == 0 and j == 1:
                        print(bank_sheet.cell_value(i,j),end="\t\t")
                        continue

                    if j == 2:
                        continue

                    if i > 0 and j == 3:
                        print("Rs. {:.2f}".format(float(bank_sheet.cell_value(i,j))),end="\t")
                        continue

                    print(bank_sheet.cell_value(i,j),end="\t")

                print()

#                 asking admin to enter the username of the user which admin wants to remove
            user = input("Enter username of the user which you want to remove : ")

#             if username present in user_details key then it delete the key by using del keyword
            if user in user_details:
                del user_details[user]

#             after removing the user from user_details dictionary it updates the excel sheets
            self.update()
            self.update_data()
        
#         printing confirmation message
            print("User Removed Successfully")

#             taking input from user 
            opt = int(input("Press 0 to return to the main menu else press any other key to signout : "))

#             if user enters 0 then it returns to the admin_options functions
            if opt == 0:
                return self.admin_options(username)

#         else returns to the main function
            else:
                return main()
                
        except KeyboardInterrupt:
            print("System Stopped by backend")
        
        except SystemExit:
            print("Exitted from system")
            
#         if any error arise then it return to the main function
        except:
            return main()
    
    
    
#     this function exits from the program
    def Exit_from_system(self):
        
        '''this function exits from the program by using sys.exit() function and it arise Systemexit error to handling 
        this error i used exception handling and it exitted from the program by giving message and if user press stop
        button then keyboarinterrupt error arise this error also handled by exception handling'''
        
        try:
            sys.exit()
            print("Exitted from program")
            
        except KeyboardInterrupt:
            print("System Stopped by backend")
            
        except SystemExit:
            print("Exitted from program")
    
    
    
#     this function prints user details
    def See_user_details(self , username):
        
        '''this function prints deatils all the present user this fuction can only exceeses by admin
        this function takes no parameter and also use exception handling this function not prints
        pin because pin is secret information'''
        
        try:
            
#             calling required global variables
            global bank , bank_sheet , user_details
            
#             iterating using for to print the user details
            for i in range(bank_sheet.nrows):
                
                for j in range(bank_sheet.ncols):
                    
                    if i == 0 and j == 1:
                        print(bank_sheet.cell_value(i,j),end="\t\t")
                        continue
                        
                    if j == 2:
                        continue
                        
                    if i > 0 and j == 3:
                        print("Rs. {:.2f}".format(float(bank_sheet.cell_value(i,j))),end="\t")
                        continue
                        
                    print(bank_sheet.cell_value(i,j),end="\t")
                    
                print()

#             prints confirmation message
            print("User details printed successfully.")
            
#             asking input to the user
            opt = int(input("Press 0 to return to the main menu else press any other key to signout : "))
            
#             if user enters 0 this means user wants to go back and it calls admin_options function
            if opt == 0:
                return self.admin_options(username)
            
#             else user wants to sign out and it directly return to to main function
            else:
                return main()
           
#             handling exception this exception occurs if user press stop button
        except KeyboardInterrupt:
            print("System Stopped by backend")
        
#         this exception occurs if any function use sys.exit() function
        except SystemExit:
            print("Exitted from system")
          
        
#         last exception occurs this means that user wants to sign out and it return to the main function
        except:
            return main()
            
            
       
    
#     sign out function
    def Sign_out(self):
        
        '''this function helps user to sign out and returns to the main function after giving 
        message to the user'''
        
        print("Sign Out Successfully")
        sleep(2)
        return main()
        
        

        
# this is main function which creates object of class ATM and call required functions
def main():
    
#     calling global variables which is useful
    global bank
    global bank_sheet
    global user_details
    global admin_details
        
    
#     printing information for the user
    print("1. Already a user")
    print("2. New User")
    
    
#     using exception handling 
    try:
        
#         creating object of class ATM
        atm = ATM()
        
#         taking input from user which option user wants to choose
        opt = int(input("Choose from above options : "))

#         if user enters 1 this means that user is existing customer
        if opt == 1:
            
#             asking user to enter username
            username = input("Enter your username : ")
            
#             if username equals to admin_username this means that user is admin and according to that main function calls
#             function
            if username in admin_details:
        
#                 calling admin_pin function where 1st argument is admin_password and 2nd argument is number of triesinitially
#                 tries equals to 1
                atm.admin_pin(username , admin_details[username] , 1)
             
#             2nd condition is if username present in user_details
            elif username in user_details:
        
#                 calling pin function where 1st argument is username and 2nd argument is number of tries 
#                 initially tries equals to 1
                atm.pin(username , 1)
                
        
#             '''3rd condition is username not present in user_details and username not equals to admin_username
#             this means that user entered wrong username this give warning to the user and returning to the main
#             function'''
            elif username not in user_details:
                print("Enter correct username")
                return main()
        
#         else if user choose 2nd option it means that new user 
        elif opt == 2:
        
            '''it calls Add_user user which takes only 1 argument which represents that admin creating user or not
            if admin creating user then we pass argument as True but here user creating new account so we pass
            argument as False'''
            atm.Add_user(False)
           
        
#         last condintion if user enters wrong input then this give warning and return to main function
        else:
            print("Choose correct option")
            return main()
        
#     handling KeyboardInterrupt error this means that user presses stop button
    except KeyboardInterrupt:
        print("System Stopped by backend")
       
    
#     handling SystemExit error this means that at some function i use sys.exit() which arise error
    except SystemExit:
        print("Exitted from system")
     
    
#     last error arise it means that in above input user not entered integer and by giving warning
#     it returns to main function
    except:
        print("Input must be an integer")
        return main()
    

# calling main function
if __name__ == "__main__":
    main()

1. Already a user
2. New User
Choose from above options : 1
Enter your username : admin
Enter correct username
1. Already a user
2. New User
Choose from above options : admin1
Input must be an integer
1. Already a user
2. New User
Choose from above options : 1
Enter your username : admin1
Enter Pin : ········
1. See user details
2. Add user
3. Remove user
4. Admin new pin generate
5. Sign out
6. Exit from system
Enter from above options : 1
Username	Name		Amount	
Aryan1234	Aryan Jain	Rs. 10001.00	
Nitin5678	Nitin Jain	Rs. 10000.00	
Rohit9123	Rohit Kumar	Rs. 10000.00	
Ronak4567	Ronak Jain	Rs. 10000.00	
Mayank8912	Mayank Pant	Rs. 10000.00	
User details printed successfully.
System Stopped by backend
